In [ ]:
!pip install -q super-gradients==3.1.3 filterpy==1.1.0

In [ ]:
import cv2
import torch
from super_gradients.training import models
import numpy as np
import math

In [ ]:
device=torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model=models.get('yolo_nas_s', pretrained_weights="coco").to(device)

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

In [ ]:
cap = cv2.VideoCapture("/content/vehicle-counting.mp4")

In [ ]:
names = [ 'person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
                'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
                'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
                'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor',
                'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                'sink', 'refrigerator', 'book', 'clock', 'vase',
                'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out=cv2.VideoWriter('output1.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 20, (width, height))

In [ ]:
!gdown "https://drive.google.com/uc?id=11ZSZcG-bcbueXZC3rN08CM0qqX3eiHxf&confirm=t"
! unzip /content/deep_sort_pytorch.zip

In [ ]:
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort

In [ ]:
cfg_deep = get_config()
cfg_deep.merge_from_file("/content/deep_sort_pytorch/configs/deep_sort.yaml")
deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT,
                    max_dist=cfg_deep.DEEPSORT.MAX_DIST, min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE,
                    nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                    max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE,
                    max_age=cfg_deep.DEEPSORT.MAX_AGE, n_init=cfg_deep.DEEPSORT.N_INIT,
                    nn_budget=cfg_deep.DEEPSORT.NN_BUDGET,
                    use_cuda=True)

In [ ]:
palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

def compute_color_for_labels(label):
  """
  Simple function that adds fixed color depending on the class
  """
  if label == 0: #person
      color = (85,45,255)
  elif label == 2: # Car
      color = (222,82,175)
  elif label == 3:  # Motobike
      color = (0, 204, 255)
  elif label == 5:  # Bus
      color = (0, 149, 255)
  else:
      color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
  return tuple(color)

In [ ]:
def draw_boxes(img, bbox, identities=None, categories=None, names=None, offset=(0,0)):
    for i, box in enumerate(bbox):
        x1, y1,  x2, y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[0]
        y2 += offset[0]
        cat = int(categories[i]) if categories is not None else 0
        id = int(identities[i]) if identities is not None else 0
        cv2.rectangle(img, (x1, y1), (x2, y2), color= compute_color_for_labels(cat),thickness=2, lineType=cv2.LINE_AA)
        label = str(id) + ":" + names[cat]
        (w,h), _ = cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)
        t_size=cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)[0]
        c2=x1+t_size[0], y1-t_size[1]-3
        cv2.rectangle(frame, (x1, y1), c2, color=compute_color_for_labels(cat), thickness=-1, lineType=cv2.LINE_AA)
        cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    return img

In [ ]:
output=cv2.VideoWriter('output2.avi',
                       cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 20,
                        (width, height))
while True:
  xywh_bboxs = []
  confs = []
  oids = []
  ret, frame = cap.read()
  if ret:
    result = list(model.predict(frame, conf=0.5))[0]
    bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
    confidences = result.prediction.confidence
    labels = result.prediction.labels.tolist()
    for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
        bbox = np.array(bbox_xyxy)
        x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        conf = math.ceil((confidence*100))/100
        cx, cy = int((x1+x2)/2), int((y1+y2)/2)
        bbox_width = abs(x1-x2)
        bbox_height = abs(y1-y2)
        xcycwh = [cx, cy, bbox_width, bbox_height]
        xywh_bboxs.append(xcycwh)
        confs.append(conf)
        oids.append(int(cls))
    xywhs = torch.tensor(xywh_bboxs)
    confss= torch.tensor(confs)
    outputs = deepsort.update(xywhs, confss, oids, frame)
    if len(outputs)>0:
        bbox_xyxy = outputs[:,:4]
        identities = outputs[:, -2]
        object_id = outputs[:, -1]
        draw_boxes(frame, bbox_xyxy, identities, object_id , names=names)
    output.write(frame)
  else:
      break